In [1]:
import psycopg2
import pandas as pd
import datacompy
import pyodbc
import useful_functions as use

In [2]:
conn = psycopg2.connect(
    host='oneview-dev-db.cwp4vd8mllvz.us-east-1.rds.amazonaws.com',
    database='oneview_content',
    user='datalabs',
    password='DataLabs123')

OperationalError: could not translate host name "oneview-dev-db.cwp4vd8mllvz.us-east-1.rds.amazonaws.com" to address: Unknown host


In [5]:
ov_program_sql = '''
    SELECT * FROM ONEVIEW.RESIDENCY_PROGRAM
    '''
ov_program = pd.read_sql_query(ov_program_sql, conn)

In [7]:
program = pd.read_csv('../../Data/MasterfileCore/ProgramInformation.txt', sep='|', error_bad_lines=False, encoding='latin')
program = program[program.pgm_activity_code==0]

b'Skipping line 5168: expected 55 fields, saw 59\nSkipping line 5183: expected 55 fields, saw 106\n'


In [11]:
program_columns = {
    'pgm_id':'id',
    'pgm_spec':'specialty',
    'pgm_inst_control_type':'institution_control',
    'pgm_sequence_number':'sequence_number',
    'pgm_federal_code':'federal_code',
    'pgm_region_code':'region_code',    
    'prm_acgme_accred_ind':'acgme_accredited',
    'pgm_name':'name',
    'pgm_web_address':'web_address',
    'pgm_contact_dir_ind':'contact_director',
    'pgm_accred_status':'accreditation_status',
    'pgm_accred_eff_dt':'accreditation_effective_date',
    'pgm_init_accred_dt':'initial_accreditation_date', 
    'pgm_accred_length':'accreditation_length',
    'pgm_yrs_offered':'duration',
    'pgm_gov_affil_ind':'govenrmnet_affiliated',
    'pgm_gme_eq_yr':'graduate_medical_education_equivalent_years',
    'pgm_prelim_yrs_req_ind':'preliminary_years_required',
    'pgm_prelim_pos_offered':'preliminary_positions_offered',
    'pgm_type':'type',
    'pgm_chg_size':'max_residents_accepted_increase',
    'pgm_%@_prim_site':'percent_at_primary_site',
    'pgm_prim_site':'primary_site',
    'pgm_core_pgm_id':'core_program',
    'official_addr_ind':'official_address',
    'pgm_sf_match':'uses_sf_match',
    'pgm_oth_match_ind':'other_match_indicator',
    'pgm_oth_match':'other_match',
    'pgm_addit_educ_accred_length':'additional_education_accreditation_length',
    'last_upd_dt':'last_update_date',
    'last_upd_type':'last_update_type',
    'AOA_Ind':'american_osteopathic_association_indicator', 
    'AOA_Program_ID':'american_osteopathic_association_indicator_program_id',
    'Osteopathic_principles':'osteopathic_principles'
    
}

In [13]:
program = program.rename(columns=program_columns)

In [14]:
program_compare = datacompy.Compare(ov_program, program, join_columns='id', df1_name = 'oneview', df2_name = 'meri')

In [15]:
print(program_compare.report())

DataComPy Comparison
--------------------

DataFrame Summary
-----------------

  DataFrame  Columns   Rows
0   oneview       43  11454
1  datamart       55  12226

Column Summary
--------------

Number of columns in common: 32
Number of columns in oneview but not in datamart: 11
Number of columns in datamart but not in oneview: 23

Row Summary
-----------

Matched on: id
Any duplicates on match values: No
Absolute Tolerance: 0
Relative Tolerance: 0
Number of rows in common: 11,454
Number of rows in oneview but not in datamart: 0
Number of rows in datamart but not in oneview: 772

Number of rows with some compared columns unequal: 11,454
Number of rows with all compared columns equal: 0

Column Comparison
-----------------

Number of columns compared with some values unequal: 15
Number of columns compared with all values equal: 17
Total number of values which compare unequal: 86,024

Columns with Unequal Values or Types
------------------------------------

                            

In [100]:
inst = pd.read_csv('../../Data/MasterfileCore/ProgramInstitution.txt', sep='|', encoding='latin')
insti = inst[inst.affiliation_type=='S']

In [101]:
institution = pd.read_csv('../../Data/MasterfileCore/InstitutionInformation.txt', sep='|', error_bad_lines=False)

b'Skipping line 601: expected 14 fields, saw 18\n'


In [102]:
addresses = pd.read_csv('../../Data/MasterfileCore/ProgramAddress.txt', sep='|', error_bad_lines=False, encoding='latin')

In [16]:
personnel = pd.read_csv('../../Data/MasterfileCore/ProgramPersonnel.txt', sep='|', encoding='latin', error_bad_lines=False)

b'Skipping line 8397: expected 35 fields, saw 36\n'
b'Skipping line 31221: expected 35 fields, saw 52\n'
b'Skipping line 51840: expected 35 fields, saw 57\nSkipping line 51876: expected 35 fields, saw 44\n'
C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (0,2,9,10,11,12,14,16,27,29,31) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [25]:
personnel = personnel.rename(columns={'pers_name_sfx':'suffix_name', 'pers_name_last':'last_name',
       'pers_name_first':'first_name', 'pers_name_mid':'middle_name', 'pers_deg1':'degree_1', 'pers_deg2':'degree_2',
       'pers_deg3':'degree_3', 'last_upd_dt':'last_update_date'})

In [54]:
personnel['program']=personnel.pgm_id

In [27]:
personnel = personnel[personnel.pers_type=='D']

In [53]:
personnel['id'] = personnel['pgm_id'].astype(str) + personnel['aamc_id'].astype(str)

In [43]:
def fix_id(num):
    num = str(num).strip().replace('.0', '')
    num = ''.join(filter(str.isdigit, num))
    if len(num) == 9:
        num = '0' + num
    return num
personnel['pgm_id']=[fix_id(x) for x in personnel.pgm_id]

In [52]:
def fix_aamc_id(num):
    num = str(num).strip().replace('.0', '')
    num = ''.join(filter(str.isdigit, num))
    while len(num) < 8:
        num = '0' + num
    return num
personnel['aamc_id']=[fix_aamc_id(x) for x in personnel.aamc_id]

In [60]:
test = personnel[personnel.program.isin(program.id)]

In [77]:
personnel_compare = datacompy.Compare(ov_personnel, test, join_columns='id', df1_name = 'oneview', df2_name = 'meri')

In [78]:
print(personnel_compare.report())

DataComPy Comparison
--------------------

DataFrame Summary
-----------------

  DataFrame  Columns   Rows
0   oneview       10  11450
1      meri       37  12222

Column Summary
--------------

Number of columns in common: 10
Number of columns in oneview but not in meri: 0
Number of columns in meri but not in oneview: 27

Row Summary
-----------

Matched on: id
Any duplicates on match values: No
Absolute Tolerance: 0
Relative Tolerance: 0
Number of rows in common: 11,450
Number of rows in oneview but not in meri: 0
Number of rows in meri but not in oneview: 772

Number of rows with some compared columns unequal: 11,450
Number of rows with all compared columns equal: 0

Column Comparison
-----------------

Number of columns compared with some values unequal: 6
Number of columns compared with all values equal: 4
Total number of values which compare unequal: 48,292

Columns with Unequal Values or Types
------------------------------------

             Column oneview dtype meri dtype  #

In [65]:
ov_personnel.id

0        040213116812326849
1        041253101613518237
2        041260402411340305
3        041261302314201750
4        041270400110044301
                ...        
11445    999540014613994247
11446    999550024810121303
11447    999560014710704136
11448    999560014810247160
11449    999560018415091268
Name: id, Length: 11450, dtype: object

In [74]:
def fix_personnel_id(num):
    num = str(num).strip().replace('.0', '')
    num = ''.join(filter(str.isdigit, num))
    while len(num) < 18:
        num =  num + '0'
    return num
ov_personnel['id'] = [fix_personnel_id(x) for x in ov_personnel.id]

In [82]:
resident_link = pd.read_csv('../../Data/MasterfileCore/RESIDENT_LINK.csv')

In [84]:
resident_link['program']=[fix_id(x) for x in resident_link.program]

In [94]:
resident_link['medical_education_number'] = use.fix_me(resident_link.medical_education_number)

In [96]:
pd.merge(resident_link, ov_physician, on = list(resident_link.columns), how='left')

,medical_education_number,program,id
0,00102000056,2202122109,220212210900102000056
1,00102000595,4450421028,445042102800102000595
2,00102010647,1554812150,155481215000102010647
3,00102020821,1401221107,140122110700102020821
4,00102030966,0401121036,040112103600102030966
...,...,...,...
8837,04813800158,9994800133,NaN
8838,03601640421,9994800168,NaN
8839,03501020833,9994900142,NaN
8840,05605060601,9995100193,NaN


In [105]:
institution

,ins_id,ins_control_type,ins_type,ins_affiliation_type,ins_name,ins_name_change_dt,ins_old_name,ins_activity_status,ins_prog_survey_coord,add_date,survey_cycle,last_upd_dt,last_upd_type,last_upd_id
0,010004,23.0,N22,A,USA Strada Patient Care Center,NaN,NaN,0,NaN,10/28/2019,2019.0,NaN,NaN,NaN
1,010005,23.0,N22,A,West Mobile Medical Group,NaN,NaN,0,NaN,8/3/2020,2020.0,NaN,NaN,NaN
2,010006,23.0,H01,A,Brewton Medical Center,NaN,NaN,0,NaN,8/3/2020,2020.0,NaN,NaN,NaN
3,010008,23.0,N22,A,Tennessee River Dermatology,NaN,NaN,0,NaN,10/15/2019,2019.0,NaN,NaN,NaN
4,010010,23.0,N22,A,Felix Morris MD LLC Office,NaN,NaN,0,NaN,10/15/2019,2019.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7177,570003,16.0,H01,A,Ivinson Memorial Hospital,NaN,NaN,0,NaN,5/28/2019,2019.0,NaN,NaN,NaN
7178,570100,23.0,H01,A,Wyoming Medical Center,8/1/1986,NaN,0,N,NaN,1997.0,NaN,NaN,NaN
7179,570106,13.0,H01,A,Cheyenne Regional Medical Center,2/2/2007,United Medical Center,0,N,NaN,2006.0,2/2/2007,A,cmathews
7180,578001,45.0,H01,A,Cheyenne Veterans Affairs Medical Center,NaN,NaN,0,NaN,6/6/1997,1996.0,NaN,NaN,NaN


In [110]:
def fix_id_2(num):
    num = str(num).strip().replace('.0', '')
    num = ''.join(filter(str.isdigit, num))
    if len(num) == 5:
        num = '0' + num
    return num
insti['id'] = [fix_id_2(x) for x in insti.ins_id]
institution['id'] = [fix_id_2(x) for x in institution.ins_id]
meri_institution = pd.merge(insti, institution, on='id')

C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [115]:
meri_institution = meri_institution.rename(columns={'affiliation_type':'affiliation','ins_name':'name','last_upd_dt_y':'last_update_date'})

In [116]:
meri_institution

,pgm_id,ins_id_x,affiliation,pri_clinical_loc_ind,survey_cycle_x,last_upd_dt_x,last_upd_type_x,last_upd_id_x,id,ins_id_y,...,name,ins_name_change_dt,ins_old_name,ins_activity_status,ins_prog_survey_coord,add_date,survey_cycle_y,last_update_date,last_upd_type_y,last_upd_id_y
0,200121109,10498,S,NaN,2010.0,NaN,NaN,NaN,010498,010498,...,University of Alabama Hospital,1/1/1986,NaN,0,N,NaN,2000.0,NaN,NaN,NaN
1,400121010,10498,S,1.0,1996.0,NaN,NaN,NaN,010498,010498,...,University of Alabama Hospital,1/1/1986,NaN,0,N,NaN,2000.0,NaN,NaN,NaN
2,410121034,10498,S,1.0,2007.0,NaN,NaN,NaN,010498,010498,...,University of Alabama Hospital,1/1/1986,NaN,0,N,NaN,2000.0,NaN,NaN,NaN
3,420104018,10498,S,1.0,2012.0,NaN,NaN,NaN,010498,010498,...,University of Alabama Hospital,1/1/1986,NaN,0,N,NaN,2000.0,NaN,NaN,NaN
4,430104001,10498,S,1.0,2012.0,NaN,NaN,NaN,010498,010498,...,University of Alabama Hospital,1/1/1986,NaN,0,N,NaN,2000.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12007,4401100439,118126,S,NaN,2015.0,NaN,NaN,NaN,118126,118126,...,Largo Medical Center,NaN,NaN,0,NaN,6/25/2015,2018.0,3/19/2019,M,Meded
12008,4405400350,548096,S,NaN,2018.0,NaN,NaN,NaN,548096,548096,...,St Joseph's Medical Center,2/27/2019,CHI Franciscan Health-St Joseph's Medical Center,0,NaN,6/9/2014,2014.0,NaN,NaN,NaN
12009,4460544003,59585,S,1.0,2012.0,NaN,NaN,NaN,059585,059585,...,John Wayne Cancer Institute at St John's Healt...,NaN,NaN,0,NaN,2/28/2013,2013.0,NaN,NaN,NaN
12010,9992400246,240448,S,1.0,2003.0,NaN,NaN,NaN,240448,240448,...,Newton-Wellesley Hospital,NaN,NaN,0,N,NaN,1998.0,NaN,NaN,NaN


In [149]:
inst_compare = datacompy.Compare(ov_institution, institution[institution.ins_affiliation_type=='S'], join_columns='id', df1_name = 'oneview', df2_name = 'meri')

In [150]:
print(inst_compare.report())

DataComPy Comparison
--------------------

DataFrame Summary
-----------------

  DataFrame  Columns  Rows
0   oneview        4   807
1      meri       18   806

Column Summary
--------------

Number of columns in common: 4
Number of columns in oneview but not in meri: 0
Number of columns in meri but not in oneview: 14

Row Summary
-----------

Matched on: id
Any duplicates on match values: No
Absolute Tolerance: 0
Relative Tolerance: 0
Number of rows in common: 806
Number of rows in oneview but not in meri: 1
Number of rows in meri but not in oneview: 0

Number of rows with some compared columns unequal: 806
Number of rows with all compared columns equal: 0

Column Comparison
-----------------

Number of columns compared with some values unequal: 1
Number of columns compared with all values equal: 3
Total number of values which compare unequal: 806

Columns with Unequal Values or Types
------------------------------------

             Column oneview dtype meri dtype  # Unequal  Max D

In [120]:
inst_compare.df1_unq_rows

,id,name,affiliation,last_update_date
45,059795,Sierra View Medical Center,S,2019-01-25
47,059810,Loma Linda University Medical Center-Murrieta,S,1970-01-01
57,410063,Tower Health,S,1970-01-01
134,479580,HCA Healthcare/TriStar Nashville,S,1970-01-01
135,479581,HCA Healthcare/TriStar Southern Hills Medical ...,S,1970-01-01
158,488244,Baylor All Saints Medical Center-Fort Worth,S,2011-01-28
171,489905,HCA Medical City Healthcare UNT-TCU Graduate M...,S,1970-01-01
211,548201,Elson S Floyd College of Medicine Washington S...,S,1970-01-01
236,389953,OhioHealth,S,1970-01-01
241,040025,Conway Regional Health System,S,1970-01-01


In [123]:
meri_institution[meri_institution.name=='North Alabama Medical Center']

,pgm_id,ins_id_x,affiliation,pri_clinical_loc_ind,survey_cycle_x,last_upd_dt_x,last_upd_type_x,last_upd_id_x,id,ins_id_y,...,name,ins_name_change_dt,ins_old_name,ins_activity_status,ins_prog_survey_coord,add_date,survey_cycle_y,last_update_date,last_upd_type_y,last_upd_id_y


In [148]:
institution['affiliation']=institution.ins_affiliation_type
institution['last_update_date']=institution.last_upd_dt

In [130]:
ov_institution.groupby('affiliation').count()

,id,name,last_update_date
affiliation,,,
,1,1,1
S,806,806,806


In [132]:
institution['name'] = institution.ins_name

In [142]:
institution[institution.ins_affiliation_type=='S']

,ins_id,ins_control_type,ins_type,ins_affiliation_type,ins_name,ins_name_change_dt,ins_old_name,ins_activity_status,ins_prog_survey_coord,add_date,survey_cycle,last_upd_dt,last_upd_type,last_upd_id,id,name
10,010017,16.0,H01,S,East Alabama Medical Center,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,010017,East Alabama Medical Center
28,010406,12.0,H01,S,USA Health,8/3/2020,University of South Alabama Medical Center,0,Y,NaN,2001.0,NaN,NaN,NaN,010406,USA Health
30,010498,12.0,H01,S,University of Alabama Hospital,1/1/1986,NaN,0,N,NaN,2000.0,NaN,NaN,NaN,010498,University of Alabama Hospital
38,010515,23.0,H01,S,St Vincent's East,1/17/2008,Medical Center East,0,N,NaN,2003.0,1/27/2010,M,Meded,010515,St Vincent's East
39,010519,23.0,N18,S,Brookwood Baptist Health,1/1/2016,Baptist Health System Inc,0,Y,NaN,1998.0,NaN,NaN,NaN,010519,Brookwood Baptist Health
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7098,560441,23.0,N22,S,Medical College of Wisconsin Affiliated Hospit...,12/20/2001,Medical College of Wisconsin Affiliated Hospit...,0,Y,NaN,2001.0,NaN,NaN,NaN,560441,Medical College of Wisconsin Affiliated Hospit...
7111,568020,23.0,H01,S,Aurora Health Care,NaN,NaN,0,NaN,NaN,1997.0,NaN,NaN,NaN,568020,Aurora Health Care
7115,568030,23.0,H01,S,Gundersen Lutheran Medical Foundation,NaN,Gundersen Medical Foundation,0,NaN,10/14/1998,2000.0,NaN,NaN,NaN,568030,Gundersen Lutheran Medical Foundation
7135,569501,12.0,N11,S,University of Wisconsin School of Medicine and...,12/13/2005,University of Wisconsin Medical School,0,NaN,NaN,2005.0,1/11/2006,A,cmathews,569501,University of Wisconsin School of Medicine and...


In [140]:
ov_institution

,id,name,affiliation,last_update_date
0,058112,St Joseph's Medical Center,S,1970-01-01
1,070705,Poudre Valley Hospital,S,1970-01-01
2,120497,Northside Hospital,S,1970-01-01
3,230320,St Agnes HealthCare,S,1970-01-01
4,240180,Berkshire Medical Center,S,1970-01-01
...,...,...,...,...
802,510716,Centra Health Inc,S,1970-01-01
803,518024,Carilion Clinic-Virginia Tech Carilion School ...,S,2010-07-23
804,518026,Orthopaedic Research of Virginia,S,1970-01-01
805,518027,Valley Health System,S,1970-01-01


In [151]:
ov_program

,id,specialty,institution_control,sequence_number,federal_code,region_code,name,web_address,contact_director,accreditation_status,...,address_1,address_2,address_3,city,state,zipcode,institution,primary_clinical_location,american_osteopathic_association_program,acgme_accredited
0,0401611044,040,11,44,7,3,University of Chicago Program,http://anesthesia.uchicago.edu/,True,AO,...,Univ of Chicago Med Ctr,MC4028,5841 S Maryland Ave,Chicago,IL,60544,160465,1,,True
1,0451021039,045,21,39,7,5,George Washington University Program,http://www.gwicu.com,True,CA,...,George Washington Univ Hosp,G-2092,900 23rd St NW,Washington,DC,20037,109501,0,,True
2,0802821051,080,21,51,7,4,Washington University/B-JH/SLCH Consortium Pro...,http://dermatology.wustl.edu,True,AO,...,Barnes-Jewish Hosp,Div of Dermatology,660 S Euclid Ave,St Louis,MO,63110,289502,0,,True
3,0402821166,040,21,166,7,4,St Louis University School of Medicine Program,https://www.slu.edu/medicine/anesthesiology-cr...,True,AO,...,St Louis Univ Hosp,Dept of Anesthesiology FDT-3,3635 Vista Ave,St Louis,MO,63110,289501,0,,True
4,0405111159,040,11,159,7,5,University of Virginia Medical Center Program,https://med.virginia.edu/anesthesiology/,True,AO,...,Univ of Virginia Health System,Dept of Anesthesiology,PO Box 800710,Charlottesville,VA,22908,510124,1,,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11449,4850548026,485,48,026,7,9,Children's Hospital Los Angeles Program,,False,I1,...,Children's Hosp Los Angeles,Mail Stop #114,4650 Sunset Blvd,Los Angeles,CA,90027,050344,,,True
11450,5405112095,540,12,095,7,5,Eastern Virginia Medical School Program,,True,I1,...,Eastern Virginia Med Sch,Ste 201,825 Fairfax Ave,Norfolk,VA,23507,510714,,,True
11451,7540100001,754,0,001,7,6,University of Alabama Hospital Program,http://www.uab.edu/medicine/obgyn/,True,AC,...,Univ of Alabama Hosp,176F Ste 10270,625 19th St S,Birmingham,AL,352497333,010498,,,True
11452,9995600148,999,0,148,7,3,Aurora Health Care Program,http://www.aurorahealthcare.org,True,AO,...,Aurora St Luke's Med Ctr,Med Office Bldg 3 Ste 770,2801 W Kinnickinnic River Pkwy,Milwaukee,WI,53215,568020,0,,True


In [154]:
inst

,pgm_id,ins_id,affiliation_type,pri_clinical_loc_ind,survey_cycle,last_upd_dt,last_upd_type,last_upd_id
0,200121109,10497,A,0.0,2009.0,NaN,NaN,NaN
1,200121109,10498,A,1.0,2017.0,NaN,NaN,NaN
2,200121109,10498,S,NaN,2010.0,NaN,NaN,NaN
3,200121109,18058,A,NaN,2017.0,NaN,NaN,NaN
4,200121109,18059,A,0.0,2008.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
40004,9995600184,560370,A,0.0,2015.0,NaN,NaN,NaN
40005,9995600184,560441,S,NaN,2015.0,NaN,NaN,NaN
40006,9995600184,560739,A,NaN,2015.0,NaN,NaN,NaN
40007,9995600184,568074,A,0.0,2010.0,NaN,NaN,NaN


,ins_id,ins_control_type,ins_type,ins_affiliation_type,ins_name,ins_name_change_dt,ins_old_name,ins_activity_status,ins_prog_survey_coord,add_date,survey_cycle,last_upd_dt,last_upd_type,last_upd_id,id,name,affiliation,last_update_date
0,010004,23.0,N22,A,USA Strada Patient Care Center,NaN,NaN,0,NaN,10/28/2019,2019.0,NaN,NaN,NaN,010004,USA Strada Patient Care Center,A,NaN
1,010005,23.0,N22,A,West Mobile Medical Group,NaN,NaN,0,NaN,8/3/2020,2020.0,NaN,NaN,NaN,010005,West Mobile Medical Group,A,NaN
2,010006,23.0,H01,A,Brewton Medical Center,NaN,NaN,0,NaN,8/3/2020,2020.0,NaN,NaN,NaN,010006,Brewton Medical Center,A,NaN
3,010008,23.0,N22,A,Tennessee River Dermatology,NaN,NaN,0,NaN,10/15/2019,2019.0,NaN,NaN,NaN,010008,Tennessee River Dermatology,A,NaN
4,010010,23.0,N22,A,Felix Morris MD LLC Office,NaN,NaN,0,NaN,10/15/2019,2019.0,NaN,NaN,NaN,010010,Felix Morris MD LLC Office,A,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7177,570003,16.0,H01,A,Ivinson Memorial Hospital,NaN,NaN,0,NaN,5/28/2019,2019.0,NaN,NaN,NaN,570003,Ivinson Memorial Hospital,A,NaN
7178,570100,23.0,H01,A,Wyoming Medical Center,8/1/1986,NaN,0,N,NaN,1997.0,NaN,NaN,NaN,570100,Wyoming Medical Center,A,NaN
7179,570106,13.0,H01,A,Cheyenne Regional Medical Center,2/2/2007,United Medical Center,0,N,NaN,2006.0,2/2/2007,A,cmathews,570106,Cheyenne Regional Medical Center,A,2/2/2007
7180,578001,45.0,H01,A,Cheyenne Veterans Affairs Medical Center,NaN,NaN,0,NaN,6/6/1997,1996.0,NaN,NaN,NaN,578001,Cheyenne Veterans Affairs Medical Center,A,NaN


In [162]:
addresses['pgm_id']=[fix_id(x) for x in addresses.pgm_id]
address = addresses[addresses.addr_type=='D']

In [166]:
program = pd.merge(address, program, left_on='pgm_id', right_on='id')

In [167]:
ov_program.columns

Index(['id', 'specialty', 'institution_control', 'sequence_number',
       'federal_code', 'region_code', 'name', 'web_address',
       'contact_director', 'accreditation_status',
       'accreditation_effective_date', 'initial_accreditation_date',
       'accreditation_length', 'duration', 'government_affiliated',
       'graduate_medical_education_equivalent_years',
       'preliminary_years_required', 'preliminary_positions_offered', 'type',
       'max_residents_accepted_increase', 'percent_at_primary_site',
       'primary_site', 'core_program', 'medical_records', 'official_address',
       'uses_sfmatch', 'other_match_indicator', 'other_match',
       'additional_education_accreditation_length', 'last_update_date',
       'last_update_type', 'american_osteopathic_association_indicator',
       'osteopathic_principles', 'address_1', 'address_2', 'address_3', 'city',
       'state', 'zipcode', 'institution', 'primary_clinical_location',
       'american_osteopathic_association_prog

In [175]:
inst['ins_id'] = [fix_id_2(x) for x in inst.ins_id]

In [180]:
insti['ins_id'] = [fix_id_2(x) for x in insti.ins_id]

C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [184]:
institution[institution.ins_affiliation_type=='S']

,ins_id,ins_control_type,ins_type,ins_affiliation_type,ins_name,ins_name_change_dt,ins_old_name,ins_activity_status,ins_prog_survey_coord,add_date,survey_cycle,last_upd_dt,last_upd_type,last_upd_id,id,name,affiliation,last_update_date
10,010017,16.0,H01,S,East Alabama Medical Center,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,010017,East Alabama Medical Center,S,NaN
28,010406,12.0,H01,S,USA Health,8/3/2020,University of South Alabama Medical Center,0,Y,NaN,2001.0,NaN,NaN,NaN,010406,USA Health,S,NaN
30,010498,12.0,H01,S,University of Alabama Hospital,1/1/1986,NaN,0,N,NaN,2000.0,NaN,NaN,NaN,010498,University of Alabama Hospital,S,NaN
38,010515,23.0,H01,S,St Vincent's East,1/17/2008,Medical Center East,0,N,NaN,2003.0,1/27/2010,M,Meded,010515,St Vincent's East,S,1/27/2010
39,010519,23.0,N18,S,Brookwood Baptist Health,1/1/2016,Baptist Health System Inc,0,Y,NaN,1998.0,NaN,NaN,NaN,010519,Brookwood Baptist Health,S,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7098,560441,23.0,N22,S,Medical College of Wisconsin Affiliated Hospit...,12/20/2001,Medical College of Wisconsin Affiliated Hospit...,0,Y,NaN,2001.0,NaN,NaN,NaN,560441,Medical College of Wisconsin Affiliated Hospit...,S,NaN
7111,568020,23.0,H01,S,Aurora Health Care,NaN,NaN,0,NaN,NaN,1997.0,NaN,NaN,NaN,568020,Aurora Health Care,S,NaN
7115,568030,23.0,H01,S,Gundersen Lutheran Medical Foundation,NaN,Gundersen Medical Foundation,0,NaN,10/14/1998,2000.0,NaN,NaN,NaN,568030,Gundersen Lutheran Medical Foundation,S,NaN
7135,569501,12.0,N11,S,University of Wisconsin School of Medicine and...,12/13/2005,University of Wisconsin Medical School,0,NaN,NaN,2005.0,1/11/2006,A,cmathews,569501,University of Wisconsin School of Medicine and...,S,1/11/2006


In [186]:
ov_program

,id,specialty,institution_control,sequence_number,federal_code,region_code,name,web_address,contact_director,accreditation_status,...,address_1,address_2,address_3,city,state,zipcode,institution,primary_clinical_location,american_osteopathic_association_program,acgme_accredited
0,0401611044,040,11,44,7,3,University of Chicago Program,http://anesthesia.uchicago.edu/,True,AO,...,Univ of Chicago Med Ctr,MC4028,5841 S Maryland Ave,Chicago,IL,60544,160465,1,,True
1,0451021039,045,21,39,7,5,George Washington University Program,http://www.gwicu.com,True,CA,...,George Washington Univ Hosp,G-2092,900 23rd St NW,Washington,DC,20037,109501,0,,True
2,0802821051,080,21,51,7,4,Washington University/B-JH/SLCH Consortium Pro...,http://dermatology.wustl.edu,True,AO,...,Barnes-Jewish Hosp,Div of Dermatology,660 S Euclid Ave,St Louis,MO,63110,289502,0,,True
3,0402821166,040,21,166,7,4,St Louis University School of Medicine Program,https://www.slu.edu/medicine/anesthesiology-cr...,True,AO,...,St Louis Univ Hosp,Dept of Anesthesiology FDT-3,3635 Vista Ave,St Louis,MO,63110,289501,0,,True
4,0405111159,040,11,159,7,5,University of Virginia Medical Center Program,https://med.virginia.edu/anesthesiology/,True,AO,...,Univ of Virginia Health System,Dept of Anesthesiology,PO Box 800710,Charlottesville,VA,22908,510124,1,,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11449,4850548026,485,48,026,7,9,Children's Hospital Los Angeles Program,,False,I1,...,Children's Hosp Los Angeles,Mail Stop #114,4650 Sunset Blvd,Los Angeles,CA,90027,050344,,,True
11450,5405112095,540,12,095,7,5,Eastern Virginia Medical School Program,,True,I1,...,Eastern Virginia Med Sch,Ste 201,825 Fairfax Ave,Norfolk,VA,23507,510714,,,True
11451,7540100001,754,0,001,7,6,University of Alabama Hospital Program,http://www.uab.edu/medicine/obgyn/,True,AC,...,Univ of Alabama Hosp,176F Ste 10270,625 19th St S,Birmingham,AL,352497333,010498,,,True
11452,9995600148,999,0,148,7,3,Aurora Health Care Program,http://www.aurorahealthcare.org,True,AO,...,Aurora St Luke's Med Ctr,Med Office Bldg 3 Ste 770,2801 W Kinnickinnic River Pkwy,Milwaukee,WI,53215,568020,0,,True


In [187]:
program.columns

Index(['pgm_id', 'addr_type', 'addr1', 'addr2', 'addr3', 'city', 'state',
       'Zip+4', 'survey_cycle_x', 'last_upd_dt', 'last_upd_type',
       'last_upd_id_x', 'id', 'specialty', 'pgm_state', 'institution_control',
       'sequence_number', 'federal_code', 'region_code', 'pgm_activity_code',
       'acgme_accredited', 'name', 'web_address', 'pgm_old_name',
       'pgm_name_change_date', 'contact_director', 'accreditation_status',
       'accreditation_effective_date', 'initial_accreditation_date',
       'accreditation_length', 'duration', 'govenrmnet_affiliated',
       'graduate_medical_education_equivalent_years',
       'preliminary_years_required', 'pgm_prelim_yrs_req',
       'preliminary_positions_offered', 'pgm_other_match_ind', 'type',
       'max_residents_accepted_increase', 'pgm_chg_type',
       'percent_at_primary_site', 'primary_site', 'pgm_prim_site_comments',
       'pgm_add_date', 'core_program', 'medical_records',
       'pgm_web_address_case_sen', 'eras', 'eras2

In [200]:
ov_program

,id,specialty,institution_control,sequence_number,federal_code,region_code,name,web_address,contact_director,accreditation_status,...,address_1,address_2,address_3,city,state,zipcode,institution,primary_clinical_location,american_osteopathic_association_program,acgme_accredited
0,0401611044,040,11,44,7,3,University of Chicago Program,http://anesthesia.uchicago.edu/,True,AO,...,Univ of Chicago Med Ctr,MC4028,5841 S Maryland Ave,Chicago,IL,60544,160465,1,,True
1,0451021039,045,21,39,7,5,George Washington University Program,http://www.gwicu.com,True,CA,...,George Washington Univ Hosp,G-2092,900 23rd St NW,Washington,DC,20037,109501,0,,True
2,0802821051,080,21,51,7,4,Washington University/B-JH/SLCH Consortium Pro...,http://dermatology.wustl.edu,True,AO,...,Barnes-Jewish Hosp,Div of Dermatology,660 S Euclid Ave,St Louis,MO,63110,289502,0,,True
3,0402821166,040,21,166,7,4,St Louis University School of Medicine Program,https://www.slu.edu/medicine/anesthesiology-cr...,True,AO,...,St Louis Univ Hosp,Dept of Anesthesiology FDT-3,3635 Vista Ave,St Louis,MO,63110,289501,0,,True
4,0405111159,040,11,159,7,5,University of Virginia Medical Center Program,https://med.virginia.edu/anesthesiology/,True,AO,...,Univ of Virginia Health System,Dept of Anesthesiology,PO Box 800710,Charlottesville,VA,22908,510124,1,,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11449,4850548026,485,48,026,7,9,Children's Hospital Los Angeles Program,,False,I1,...,Children's Hosp Los Angeles,Mail Stop #114,4650 Sunset Blvd,Los Angeles,CA,90027,050344,,,True
11450,5405112095,540,12,095,7,5,Eastern Virginia Medical School Program,,True,I1,...,Eastern Virginia Med Sch,Ste 201,825 Fairfax Ave,Norfolk,VA,23507,510714,,,True
11451,7540100001,754,0,001,7,6,University of Alabama Hospital Program,http://www.uab.edu/medicine/obgyn/,True,AC,...,Univ of Alabama Hosp,176F Ste 10270,625 19th St S,Birmingham,AL,352497333,010498,,,True
11452,9995600148,999,0,148,7,3,Aurora Health Care Program,http://www.aurorahealthcare.org,True,AO,...,Aurora St Luke's Med Ctr,Med Office Bldg 3 Ste 770,2801 W Kinnickinnic River Pkwy,Milwaukee,WI,53215,568020,0,,True


In [201]:
program_2 = pd.merge(program, insti[insti.ins_id.isin(institution.ins_id)], left_on='id', right_on='program')

In [198]:
insti['program']=[fix_id(x) for x in insti.pgm_id]

C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [197]:
fix_id(200121109)

'0200121109'

In [258]:
program_compare = datacompy.Compare(ov_program, program_2, join_columns='id', df1_name = 'oneview', df2_name = 'meri')

In [205]:
program_2 = program_2.rename(columns = {'id_x':'id'})

In [259]:
print(program_compare.report())

DataComPy Comparison
--------------------

DataFrame Summary
-----------------

  DataFrame  Columns   Rows
0   oneview       43  11454
1      meri       77  11605

Column Summary
--------------

Number of columns in common: 43
Number of columns in oneview but not in meri: 0
Number of columns in meri but not in oneview: 34

Row Summary
-----------

Matched on: id
Any duplicates on match values: No
Absolute Tolerance: 0
Relative Tolerance: 0
Number of rows in common: 11,453
Number of rows in oneview but not in meri: 1
Number of rows in meri but not in oneview: 152

Number of rows with some compared columns unequal: 11,453
Number of rows with all compared columns equal: 0

Column Comparison
-----------------

Number of columns compared with some values unequal: 18
Number of columns compared with all values equal: 25
Total number of values which compare unequal: 113,808

Columns with Unequal Values or Types
------------------------------------

                                         Col

In [212]:
program[program.pgm_id=='1201611099']

,pgm_id,addr_type,addr1,addr2,addr3,city,state,Zip+4,survey_cycle_x,last_upd_dt,...,pgm_addit_length_training,pgm_addit_educ_exp_req,pgm_prev_edu_req_prior,survey_cycle_y,last_update_date,last_update_type,last_upd_id_y,american_osteopathic_association_indicator,american_osteopathic_association_indicator_program_id,osteopathic_principles


In [211]:
program

,pgm_id,addr_type,addr1,addr2,addr3,city,state,Zip+4,survey_cycle_x,last_upd_dt,...,pgm_addit_length_training,pgm_addit_educ_exp_req,pgm_prev_edu_req_prior,survey_cycle_y,last_update_date,last_update_type,last_upd_id_y,american_osteopathic_association_indicator,american_osteopathic_association_indicator_program_id,osteopathic_principles
0,0200121109,D,Children's Hosp Alabama,CPPN Ste M10,1600 7th Ave S,Birmingham,AL,35233.0,2020.0,6/10/2020,...,NaN,0.0,1.0,2020.0,6/11/2020,M,Meded,0.0,NaN,NaN
1,0200300001,D,Univ of Arizona Coll of Med-Tucson,PO Box 245030,1501 N Campbell Ave,Tucson,AZ,857245030.0,2020.0,7/2/2020,...,NaN,0.0,1.0,2020.0,7/2/2020,V,cmathews,0.0,NaN,0.0
2,0200300002,D,Mayo Clinic Arizona,Allergy/Immunology Pgm,13400 E Shea Blvd,Scottsdale,AZ,85259.0,2020.0,7/2/2020,...,NaN,NaN,1.0,2020.0,7/2/2020,V,cmathews,0.0,NaN,0.0
3,0200400001,D,Arkansas Children's Hosp,Slot 512-13,13 Children's Way,Little Rock,AR,72202.0,2020.0,6/16/2020,...,NaN,0.0,1.0,2020.0,9/10/2020,K,setzel,1.0,NaN,NaN
4,0200500131,D,UCSF Med Ctr,Box 0434 4th Fl,550 16th St,San Francisco,CA,94158.0,2020.0,7/2/2020,...,NaN,0.0,1.0,2020.0,7/2/2020,V,cmathews,0.0,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12220,9995400146,D,Madigan Army Med Ctr,MCHJ-CLM,9040A Jackson Ave,Tacoma,WA,984311100.0,2020.0,7/14/2020,...,NaN,NaN,4.0,2020.0,7/14/2020,V,cmathews,0.0,NaN,0.0
12221,9995500248,D,West Virginia Univ HSC,RCB Hlth Sciences Ctr Box 9001-A,1 Medical Center Dr,Morgantown,WV,26506.0,2020.0,6/10/2020,...,NaN,NaN,4.0,2020.0,6/10/2020,V,cmathews,0.0,NaN,0.0
12222,9995600147,D,Gundersen Med Foundation,C03-006A,1900 South Ave,La Crosse,WI,54601.0,2020.0,7/2/2020,...,NaN,NaN,4.0,2020.0,7/2/2020,V,cmathews,0.0,NaN,NaN
12223,9995600148,D,Aurora St Luke's Med Ctr,Med Office Bldg 3 Ste 770,2801 W Kinnickinnic River Pkwy,Milwaukee,WI,53215.0,2020.0,6/16/2020,...,NaN,NaN,4.0,2020.0,6/16/2020,V,cmathews,0.0,NaN,0.0


In [213]:
ov_personnel

,id,program,first_name,middle_name,last_name,suffix_name,degree_1,degree_2,degree_3,last_update_date
0,040213116812326849,0402131168,Katherine,S,Cox,,MD,,,2020-06-10
1,041253101613518237,0412531016,Anna,,Dubovoy,,MD,,,2020-07-22
2,041260402411340305,0412604024,Tjorvi,E,Perry,,MD,MMS,,2020-07-22
3,041261302314201750,0412613023,Mark,M,Smith,,MD,,,2020-07-14
4,041270400110044301,0412704001,Vanetta,L,Levesque,,MD,,,2020-07-22
...,...,...,...,...,...,...,...,...,...,...
11445,999540014613994247,9995400146,Anjali,N,Kunz,,MD,,,2020-07-14
11446,999550024810121303,9995500248,Mary,D,Warden,,MD,,,2020-06-10
11447,999560014710704136,9995600147,Lori,S,Weber,,MD,,,2020-07-02
11448,999560014810247160,9995600148,Colleen,,Nichols,,MD,,,2020-06-16


In [217]:
personnel[personnel.pgm_id.isin(ov_program.id)]

,pgm_id,pers_type,aamc_id,suffix_name,last_name,first_name,middle_name,degree_1,degree_2,degree_3,...,prev_director_sfx,acgme_verified,approval_date,why_change,subscribe,diff_name,email_notify,me.,id,program
1,0200121109,D,12247369,NaN,Atkinson,Thomas,P,MD,PhD,NaN,...,NaN,N,NaN,2.0,NaN,0.0,Y,NaN,020012110912247369,0200121109
6,0200300001,D,11236281,NaN,Carr,Tara,F,MD,NaN,NaN,...,NaN,N,NaN,2.0,NaN,0.0,Y,NaN,020030000111236281,0200300001
11,0200300002,D,10668447,NaN,Lewis,John,C,MD,NaN,NaN,...,NaN,N,NaN,2.0,NaN,0.0,Y,NaN,020030000210668447,0200300002
23,0200500131,D,14175484,NaN,Dorsey,Morna,NaN,MD,NaN,NaN,...,NaN,N,NaN,2.0,NaN,0.0,Y,NaN,020050013114175484,0200500131
28,0200511038,D,11451123,NaN,Garcia-Lloret,Maria,I,MD,MS,NaN,...,NaN,N,NaN,2.0,NaN,0.0,Y,NaN,020051103811451123,0200511038
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65585,9995400146,D,13994247,NaN,Kunz,Anjali,N,MD,NaN,NaN,...,NaN,N,NaN,1.0,NaN,0.0,Y,NaN,999540014613994247,9995400146
65590,9995500248,D,10121303,NaN,Warden,Mary,D,MD,NaN,NaN,...,NaN,N,NaN,2.0,NaN,0.0,Y,NaN,999550024810121303,9995500248
65601,9995600147,D,10704136,NaN,Weber,Lori,S,MD,NaN,NaN,...,NaN,N,NaN,2.0,NaN,0.0,Y,NaN,999560014710704136,9995600147
65606,9995600148,D,10247160,NaN,Nichols,Colleen,NaN,MD,NaN,NaN,...,NaN,N,NaN,2.0,NaN,0.0,Y,NaN,999560014810247160,9995600148


In [224]:
personnel_compare_2 = datacompy.Compare(ov_personnel, personnel, join_columns='id', df1_name = 'oneview', df2_name = 'meri')

In [225]:
print(personnel_compare_2.report())

DataComPy Comparison
--------------------

DataFrame Summary
-----------------

  DataFrame  Columns   Rows
0   oneview       10  11450
1      meri       37  12480

Column Summary
--------------

Number of columns in common: 10
Number of columns in oneview but not in meri: 0
Number of columns in meri but not in oneview: 27

Row Summary
-----------

Matched on: id
Any duplicates on match values: No
Absolute Tolerance: 0
Relative Tolerance: 0
Number of rows in common: 11,394
Number of rows in oneview but not in meri: 56
Number of rows in meri but not in oneview: 1,086

Number of rows with some compared columns unequal: 11,394
Number of rows with all compared columns equal: 0

Column Comparison
-----------------

Number of columns compared with some values unequal: 6
Number of columns compared with all values equal: 4
Total number of values which compare unequal: 48,052

Columns with Unequal Values or Types
------------------------------------

             Column oneview dtype meri dtype

In [228]:
personnel_compare_2.df1_unq_rows.to_csv('../../Data/OneView_Testing/Personnel_ID_Examples.csv', index=False)

In [234]:
print(program_compare.report())

DataComPy Comparison
--------------------

DataFrame Summary
-----------------

  DataFrame  Columns   Rows
0   oneview       43  11454
1      meri       77  11605

Column Summary
--------------

Number of columns in common: 34
Number of columns in oneview but not in meri: 9
Number of columns in meri but not in oneview: 43

Row Summary
-----------

Matched on: id
Any duplicates on match values: No
Absolute Tolerance: 0
Relative Tolerance: 0
Number of rows in common: 11,453
Number of rows in oneview but not in meri: 1
Number of rows in meri but not in oneview: 152

Number of rows with some compared columns unequal: 11,453
Number of rows with all compared columns equal: 0

Column Comparison
-----------------

Number of columns compared with some values unequal: 15
Number of columns compared with all values equal: 19
Total number of values which compare unequal: 86,018

Columns with Unequal Values or Types
------------------------------------

                                         Colu

In [231]:
inst_compare.df1_unq_rows.sample().to_csv('../../Data/OneView_Testing/Institution_Weird_Row.csv', index=False)

In [264]:
program_compare.df2_unq_rows[ov_program.columns].to_csv('../../Data/OneView_Testing/Residency_Program_Missing_Rows.csv', index=False)

In [257]:
program_2 = program_2.rename(columns={'govenrmnet_affiliated':'government_affiliated','id_y':'institution', 'uses_sf_match':'uses_sfmatch','pri_clinical_loc_ind':'primary_clinical_location','addr1':'address_1','addr2':'address_2','addr3':'address_3','american_osteopathic_association_indicator_program':'american_osteopathic_association_program', 'zip+4':'zipcode'})

In [ ]:
['government_affiliated', 'institution', 'uses_sfmatch', 'primary_clinical_location', 'address_3', 'american_osteopathic_association_program', 'address_2', 'address_1', 'zipcode']

In [252]:
program_2.columns

Index(['pgm_id_x', 'addr_type', 'address_1', 'address_2', 'addr3', 'city',
       'state', 'zipcode', 'survey_cycle_x', 'last_upd_dt_x',
       'last_upd_type_x', 'last_upd_id_x', 'id', 'specialty', 'pgm_state',
       'institution_control', 'sequence_number', 'federal_code', 'region_code',
       'pgm_activity_code', 'acgme_accredited', 'name', 'web_address',
       'pgm_old_name', 'pgm_name_change_date', 'contact_director',
       'accreditation_status', 'accreditation_effective_date',
       'initial_accreditation_date', 'accreditation_length', 'duration',
       'government_affiliated', 'graduate_medical_education_equivalent_years',
       'preliminary_years_required', 'pgm_prelim_yrs_req',
       'preliminary_positions_offered', 'pgm_other_match_ind', 'type',
       'max_residents_accepted_increase', 'pgm_chg_type',
       'percent_at_primary_site', 'primary_site', 'pgm_prim_site_comments',
       'pgm_add_date', 'core_program', 'medical_records',
       'pgm_web_address_case_sen'

In [261]:
program_compare.sample_mismatch(column="american_osteopathic_association_program").dropna()

,id,american_osteopathic_association_program_df1,american_osteopathic_association_program_df2
1722,1205522334,126330,126330.0
